In [3]:
from plyfile import PlyData, PlyElement
import numpy as np

In [4]:
plydata = PlyData.read('ply files/dragon_vrip.ply')

In [5]:
len(plydata)

2

In [6]:
print(plydata)

ply
format ascii 1.0
comment generated by ply_writer
element vertex 437645
property float x
property float y
property float z
element face 871414
property list uchar int vertex_indices
end_header


In [7]:
plydata['vertex']

PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float')), count=437645, comments=[])

In [8]:
plydata['vertex'][1]

(0.0317978, 0.056575, -0.0503375)

In [9]:
plydata['face'][0]

(array([ 2, 37,  0]),)

In [10]:
count = 0
c_x = 0;
c_y = 0;
c_z = 0;
for x in plydata['vertex']:
#     print(x[0], x[1], x[2])
#     print(str([x[0], x[1], x[2]]))
    c_x += x[0]
    c_y += x[1]
    c_z += x[2]
    count += 1

In [11]:
count

437645

In [12]:
c_x = c_x / count;
c_y = c_y / count;
c_z = c_z / count;

In [13]:
mesh_center = np.array([c_x, c_y, c_z])

In [14]:
print(mesh_center)

[-0.01156596  0.11069893 -0.00792576]


In [15]:
#calculate and store area-weighted vertex normals for future use#
vertex_normals = np.zeros((count, 3))
for x in plydata['face']:
    #get indexs of vertices in the face #
    index = x[0]
    
    #get corresponding vertices# 
    point_a = plydata['vertex'][index[0]]
    point_b = plydata['vertex'][index[1]]
    point_c = plydata['vertex'][index[2]]
    
    #use np to reconstruct to eliminate extra properties rather than xyz and vector calculation#
    t_point_a = np.array([point_a[0], point_a[1], point_a[2]])
    t_point_b = np.array([point_b[0], point_b[1], point_b[2]])
    t_point_c = np.array([point_c[0], point_c[1], point_c[2]])
    
    #construct vectors in the face#
    v1 = t_point_b - t_point_a
    v2 = t_point_c - t_point_a
    
    #calculate normal and then normalize#
    normal = np.cross(v1, v2)
    normal = normal / (np.linalg.norm(normal))


    v_t = t_point_a - mesh_center
    # if (np.dot(v_t, normal) < 0):
    #     normal =  np.negative(normal)
    # print(normal)
    
    #add normals to relavant vertex indexs 
    vertex_normals[index[0]] += normal
    vertex_normals[index[1]] += normal
    vertex_normals[index[2]] += normal

3]
[ 0.22098038  0.49594334 -0.83976656]
[ 0.17086874  0.3505615  -0.92082053]
[ 0.25718305  0.42641237 -0.8671962 ]
[ 0.3206882   0.3370026  -0.88520527]
[-0.11470478  0.9125402   0.39257246]
[0.19475031 0.92418724 0.32855782]
[0.07317969 0.88929063 0.45144975]
[-0.03258568  0.9755439   0.2173758 ]
[-0.0441755  0.9731952  0.225698 ]
[-0.04302147  0.9758188   0.21430524]
[-0.06110588  0.9898445   0.1283504 ]
[-0.06736848  0.99442124  0.081166  ]
[-0.08412204  0.995908    0.03302493]
[ 0.0034322   0.99982595 -0.01833946]
[0.02197252 0.99782896 0.06208604]
[-0.10313631  0.99155927 -0.07856876]
[-0.08521798  0.9962658  -0.01386851]
[0.52480686 0.79613155 0.30125138]
[0.5114052  0.8194155  0.25888798]
[-0.6940049  -0.5981519  -0.40071383]
[-0.773165   -0.5678759  -0.28236994]
[0.7685613  0.3453938  0.53853196]
[0.78946346 0.39616823 0.46882635]
[0.8116581  0.39516497 0.43018115]
[0.73417395 0.46400055 0.4956734 ]
[-0.28311893  0.39191934 -0.875353  ]
[-0.29498693  0.28487453 -0.9120467 ]
[

KeyboardInterrupt: 

In [16]:
print(vertex_normals)

[[-1. -2. -2.]
 [ 1. -1. -1.]
 [-2.  2. -1.]
 [ 2.  1. -2.]
 [-2. -1.  2.]
 [ 2. -2.  1.]
 [-1.  1.  1.]
 [ 1.  2.  2.]]


In [17]:
normalized_vector_normals = []
for x in vertex_normals:
    if (np.linalg.norm(x) > 0.0):
        x = x / (np.linalg.norm(x))
    normalized_vector_normals.append(x)

In [18]:
print(normalized_vector_normals)

[array([-0.33333333, -0.66666667, -0.66666667]), array([ 0.57735027, -0.57735027, -0.57735027]), array([-0.66666667,  0.66666667, -0.33333333]), array([ 0.66666667,  0.33333333, -0.66666667]), array([-0.66666667, -0.33333333,  0.66666667]), array([ 0.66666667, -0.66666667,  0.33333333]), array([-0.57735027,  0.57735027,  0.57735027]), array([0.33333333, 0.66666667, 0.66666667])]


In [19]:
# print(normalized_vector_normals[35946])

In [20]:
# print(plydata['vertex'][35878])

In [21]:
result = []
i = 0
while i < count:
    result.append([plydata['vertex'][i], normalized_vector_normals[i]])
    i += 1

In [22]:
print(result[0][1][0])

-0.3333333333333333


In [23]:
f = open("dragon.xyz", "w+")

In [24]:
f.write(str(count) + "\n")

2

In [25]:
for x in result:
    f.write(str(x[0][0]) + " ")
    f.write(str(x[0][1]) + " ")
    f.write(str(x[0][2]) + " ")
    f.write(str(x[1][0]) + " ")
    f.write(str(x[1][1]) + " ")
    f.write(str(x[1][2]) + "\n")

In [26]:
f.close()